In [ ]:
import pandas as pd
import seaborn as sns

with open("alle_gemeinden.csv") as f:
    content = f.read()

content = content.replace('","','##@#!')    #To prevent deleting required commas
content = content.replace(',','.')
content = content.replace('##@#!','","')

with open("gemeinden.csv", "w") as f:
    content = f.write(content)

dataset = pd.read_csv('gemeinden.csv', sep=';', na_values=["-"])


#dataset.head(100)
dataset.head()

In [ ]:
print(dataset.dtypes)

# überprüfe ob Datensatz vollständig ist
dataset.shape

In [ ]:
# Gemeindeliste aus Statistik.at 
# Habe ersten 2 Zeilen (und die letzte Zeile) aus dem Datensatz löschen müssen weil es Text war... 
# gibt es eine bessere Methode als manuell löschen?
dataset2 = pd.read_csv('gemliste_knz_edited.csv', sep=';')

dataset2.head()

In [ ]:
dataset2.shape

In [ ]:
dataset3 = dataset2.join(dataset.set_index('Name'), on='Gemeindename')

dataset3.head(2123)

#PROBLEM: WIEN Namen sind anders
# provisorische Lösung: ich bearbeite die Gemeindenamen im csv ... geht es auch anders?
# habe dafür edited file benutzt
# original ist auch da (gemliste_knz)

In [ ]:
dataset3.shape

In [ ]:
#df_norm = dataset.copy()
df_norm = dataset3.copy()
df_norm["Arbeitsstätten"] = df_norm["Arbeitsstätten"]/df_norm["Einwohner"]
df_norm["Beschäftigte"] = df_norm["Beschäftigte"]/df_norm["Einwohner"]
df_norm["Erbwerbstätige (15-64)"] = df_norm["Erbwerbstätige (15-64)"]/df_norm["Einwohner"]
df_norm["Anzahl an Ehepaaren"] = df_norm["Anzahl an Ehepaaren"]/df_norm["Einwohner"]

df_norm.head()

In [ ]:
features = ["Einwohner", "Arbeitsstätten", "Beschäftigte", "Alter", "Einkommen", "Erbwerbstätige (15-64)", "Grundstückspreise", "Anzahl an Ehepaaren", "Anzahl der Kinder"]
#ft_plot = sns.pairplot(dataset[features])

#ft_plot = sns.pairplot(df_norm)
ft_plot = sns.pairplot(df_norm[features])

In [ ]:
# Zum Ansehen von interessanten Sachen
# spalte="Anzahl an Ehepaaren" ## Warth df_norm.iloc[11] hat 368 Ehepaare bei 171 Einwohnern
# spalte="Beschäftigte" ## Innere Stadt Wien hat 10x so viele Beschäftigte wie Einwohner

spalte="Beschäftigte"

print("ABSOLUT")
print(dataset.loc[dataset[spalte] == dataset[spalte].max()])


print(" ")
print("RELATIV")
df_norm.loc[df_norm[spalte] == df_norm[spalte].max()]

#dataset.iloc[11]
#df_norm.iloc[11]

In [ ]:
# pandas profiling
from pandas_profiling import ProfileReport
#profile = ProfileReport(dataset, title='Pandas Profiling Report', explorative=True)

#Interessante Warnung beim normierten Datensatz: "Highly skewed"
profile = ProfileReport(df_norm, title='Pandas Profiling Report', explorative=True)
#profile.to_file("pandas_report.html")
#profile.to_notebook_iframe()

#PROBLEM: ValueError: Produced output is too large and cannot be displayed. Consider saving it to a file.
# Wie kann man das vermeiden? Kennzahl löschen/ignorieren?

In [ ]:
# Sweetviz ist vielleicht ein cooler Zusatz für die Datenanalyse. Manches wird einfacher und übersichtlicher gezeigt.
# Man kann auch zwei verschiedene Variablen miteinander vergleichen, oder ein "target" haben, ähnlich wie "groupby" von tableone.
import sweetviz as sv

my_report = sv.analyze(df_norm)
#my_report = sv.analyze(df_norm, target_feat='Beschäftigte')

# mit compare_intra kann man z.B kategorische werte miteinander vergleichen
# geht z.B mim medizin datensatz(ist hier derzeit nicht verfügbar..  aber man kann es lokal oder wo anders probieren):
#my_report = sv.compare_intra(hd_df, hd_df['sex'] == 'male', ['Male', 'Female'])

my_report.show_notebook()

#noch ein vorteil: wird viel schneller ausgeführt

In [ ]:
# Als Funktion umschreiben
def profile_hilfestellung(pandas_profile):
    import json
    from IPython.core.display import HTML

    # Profile in JSON umwandeln
    #json_data = pandas_profile.to_json()
    #pandas_profile.to_file("your_report.json")

    # JSON File importieren
    with open('your_report.json') as f:
        import_json_data = json.load(f)

    display(HTML("<h2> Hilfreiche Links: </h2>"))

    display(HTML("<p style='font-size:15px'> <a href='https://wissenschafts-thurm.de/grundlagen-der-statistik-wie-unterscheidet-man-zwischen-nominal-ordinal-und-kardinalskala/' target='_blank'>Grundlagen: Nominal-, Ordinal- und Kardinalskala</a> </p>"))

    # Man kann überprüfen ob kategorische, numerische, bool Werte vorhanden sind oder nicht
    if 'Categorical' in import_json_data['table']['types'].keys():
        display(HTML("<p style='font-size:15px'> <a href='https://en.wikipedia.org/wiki/List_of_analyses_of_categorical_data' target='_blank'>Analyse kategorischer Daten</a> </p>"))

    if 'Numeric' in import_json_data['table']['types'].keys():
        display(HTML("<p style='font-size:15px'> <a href='http://analytics.datengeschichten.at/doku.php?id=statistische_grundlagen' target='_blank'>Statistische Grundlagen</a> </p>"))

    # Schleife zum durchgehen von Werten und Infos pro Variable
    skewness_found = False
    missing_found = False


    for key, value in import_json_data['variables'].items():
        if value['n_missing'] != 0 and missing_found==False:
            display(HTML("<p style='font-size:15px'> <a href='https://www.inwt-statistics.de/blog-artikel-lesen/fehlende-werte-verstehen-und-handhaben.html' target='_blank'>Was macht man mit fehlenden Werten?</p></a>"))
            missing_found = True
        if 'skewness' in value and skewness_found==False:
            display(HTML("<p style='font-size:15px'> <a href='https://matheguru.com/stochastik/schiefe-linksschief-rechtsschief-symmetrisch.html' target='_blank'>Was bedeutet eine Skewness von " + str(value['skewness']) + " bei " + key + "?</p></a>"))
            skewness_found = True

    # Weitere allgemeine Links

    display(HTML("<p style='font-size:15px'> <a href='https://www.data-to-viz.com' target='_blank'>Visualisierungen nach/für Merkmal</a> </p>"))

    display(HTML("<p style='font-size:15px'> <a href='https://www.youtube.com/watch?v=gP-Xx26p_kc' target='_blank'>Video zur Binomial- und Normalverteilung</a> </p>"))

In [ ]:

# Funktion ausprobieren
profile_hilfestellung(profile)

In [ ]:
# kurze Übersicht: will vermutlich den normierten Datensatz benutzen
df_norm.head()

In [ ]:
#Idee: ML für Bundesländer: Gemeindekennzahl bzw Gemeindecode ignorieren, "Bundaesland" feld erstellen mit erster Ziffer (1-9).
#Oder ist es besser, Kategorien zu erstellen? Zahlen 1-9 könnten als numerische Interpretation falsch verwendet werden.

df_norm['Bundesland'] = df_norm['Gemeindekennziffer'].astype(str).str[0]
df_norm['Bundesland'] = df_norm['Bundesland'].astype(int)

df_norm.head(1000)

In [ ]:
df_norm.dtypes

In [ ]:
d_bundesland = {1:'Burgenland', 2:'Kärnten', 3:'Niederösterreich', 4:'Oberösterreich',
                5:'Salzburg', 6:'Steiermark', 7:'Tirol', 8:'Vorarlberg', 9:'Wien'}
df_norm['Bundesland'].replace(d_bundesland, inplace=True)

In [ ]:
df_norm.head()

In [ ]:
from pycaret.classification import *
exp = setup(data=df_norm, target='Bundesland', ignore_features=['Gemeindekennziffer', 'Gemeindename', 'Gemeindecode', 'PLZ des Gem.Amtes', 'weitere Postleitzahlen'], silent=True, session_id=101)

# weitere optionen: ignore_low_variance, group_features, bin_numeric_features, group_features, transformation, normalize
#    ensemble_model auch eine option? stack_models auch?

In [ ]:
#877.2s ... weniger folds? folds auf 3 probieren

#best = compare_models(fold=3, exclude=['catboost', 'xgboost', 'lightgbm'])

best = compare_models(fold=3, exclude=['lightgbm'])


In [ ]:
selected_model = create_model('rf', fold=3)

In [ ]:
tuned_model = tune_model(selected_model, fold=3)

In [ ]:
plot_model(tuned_model)

In [ ]:
interpret_model(tuned_model)

In [ ]:
evaluate_model(tuned_model)

In [ ]:
pred_holdout = predict_model(tuned_model)
pred_holdout.head()